# Packages and functions

In [222]:
!pip install unidecode

import numpy as np
import random
from random import randint
from datetime import datetime
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss, accuracy_score
import pandas as pd
import numpy as np
from time import time
from gensim.models import Word2Vec
import networkx as nx
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import os
from random import choice
from scipy.sparse import identity, diags
from unidecode import unidecode
from urllib.request import urlopen
import gzip
import pickle
from gensim.parsing.preprocessing import remove_stopwords
import re
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
from tqdm.notebook import tqdm
import requests
import io
from scipy import sparse



device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
#device = torch.device("cpu")

zsh:1: command not found: pip


In [387]:
def save_subgraph_in_file(nbr_nodes, source_path='../input_data/edgelist.txt', destination_path='../input_data/small_edgelist.txt'):
    G = nx.read_edgelist(source_path, delimiter=',', create_using=nx.Graph(), nodetype=int)
    G = G.subgraph(range(nbr_nodes))
    nx.write_edgelist(G, path=destination_path, delimiter=',')
    print(G.number_of_nodes(), 'nodes,', G.number_of_edges(), 'edges Graph extracted from', source_path[source_path.rfind('/')+1:])
    G = nx.read_edgelist(destination_path, delimiter=',', create_using=nx.Graph(), nodetype=int)
    print(G.number_of_nodes(), 'nodes,', G.number_of_edges(), 'edges Graph saved in', destination_path[destination_path.rfind('/')+1:])
    print(max(G.nodes))
    return


def read_train_val_graph(path='../input_data/edgelist.txt', val_ratio=0.1):
    #gets the data from the file on the distant server
    G = nx.read_edgelist(urlopen('https://www.lix.polytechnique.fr/~nikolentzos/files/aai/challenge/edgelist.txt'), delimiter=',', create_using=nx.Graph(), nodetype=int)
    nodes = list(G.nodes())
    edges = list(G.edges())
    n = G.number_of_nodes()
    m = G.number_of_edges()
    
    random.shuffle(edges)
    G = nx.Graph()
    G.add_edges_from(edges)
    permutation = np.random.permutation(n)
    # create a mapping from old nodes labels to new nodes labels
    mapping_permutation = dict(zip(range(n), permutation))

    G = nx.relabel_nodes(G, mapping_permutation)    
    
    edges = list(G.edges())
    nodes = list(G.nodes())


    print('Number of nodes:', n, 'number of edges:', m,'in the Complete set')

    node_to_idx = dict()
    for i, node in enumerate(nodes):
        node_to_idx[node] = i

    val_edges = list()
    G_train = G.copy()

    for edge in edges:
        if random.random() < val_ratio and edge[0] < n and edge[1] < n:
            val_edges.append(edge)
            G_train.remove_edge(edge[0], edge[1]) # We remove the val edges from the graph G

   
    #for edge in val_edges:
        

    n = G_train.number_of_nodes()
    m = G_train.number_of_edges()
    train_edges = list(G_train.edges())

    print('Number of nodes:', n, 'number of edges:', m, 'in the Training set')
    print('len(nodes)', len(nodes))

    y_val = [1]*len(val_edges)

    n_val_edges = len(val_edges)
    
    print('Creating random val_edges...')
    for i in range(n_val_edges):
        n1 = nodes[randint(0, n-1)]
        n2 = nodes[randint(0, n-1)]
        (n1, n2) = (min(n1, n2), max(n1, n2))
        while n2 >= n: #or (n1, n2) in train_edges:
            if (n1, n2) in train_edges:
                print((n1, n2), 'in train_edges:')
            n1 = nodes[randint(0, n-1)]
            n2 = nodes[randint(0, n-1)]
            (n1, n2) = (min(n1, n2), max(n1, n2))
        val_edges.append((n1, n2))

    y_val.extend([0]*(n_val_edges))
    
    ### From Giannis /!\
    val_indices = np.zeros((2,len(val_edges)))
    for i,edge in enumerate(val_edges):
        val_indices[0,i] = node_to_idx[edge[0]]
        val_indices[1,i] = node_to_idx[edge[1]]
    
    print('Returned G_train, train_edges, val_edges, y_val, nodes and node_to_idx objects')
    print('Loaded from', path[path.rfind('/')+1:], 'and with a training validation split ratio =', val_ratio)
    
    
    
    return G, G_train, train_edges, val_edges, val_indices, y_val, nodes, node_to_idx, mapping_permutation

def random_walk(G, node, walk_length):
    walk = [node]
  
    for i in range(walk_length-1):
        neibor_nodes = list(G.neighbors(walk[-1]))
        if len(neibor_nodes) > 0:
            next_node = choice(neibor_nodes)
            walk.append(next_node)
    walk = [node for node in walk] # in case the nodes are in string format, we don't need to cast into string, but if the nodes are in numeric or integer, we need this line to cast into string
    return walk


def generate_walks(G, num_walks, walk_length):
  # Runs "num_walks" random walks from each node, and returns a list of all random walk
    t = time()
    print('Start generating walks....')
    walks = list()  
    for i in range(num_walks):
        for node in G.nodes():
            walk = random_walk(G, node, walk_length)
            walks.append(walk)
        #print('walks : ', walks)
    print('Random walks generated in in {}s!'.format(round(time()-t)))
    return walks

def apply_word2vec_on_features(features, nodes, vector_size=128, window=5, min_count=0, sg=1, workers=8):
    t = time()
    print('Start applying Word2Vec...')
    wv_model = Word2Vec(vector_size=vector_size, window=window, min_count=min_count, sg=sg, workers=workers)
    wv_model.build_vocab(features)
    wv_model.train(features, total_examples=wv_model.corpus_count, epochs=5) 
    print('Word2vec model trained on features in {} min!'.format(round((time()-t)/60)))
    features_np = []
    for node in nodes:
        features_np.append(wv_model.wv[node])

    features_np = np.array(features_np)
    print(features_np.shape, 'features numpy array created in {} min!'.format(round((time()-t)/60)))
    return features_np



def normalize_adjacency(A):
    n = A.shape[0]
    A = A + identity(n)
    degs = A.dot(np.ones(n))
    inv_degs = np.power(degs, -1)
    D_inv = diags(inv_degs)
    A_hat = D_inv.dot(A)
    return A_hat

import numpy as np
import scipy.sparse as sp

def normalize_adj(adj):
    """Normalize adjacency matrix."""
    adj = sp.coo_matrix(adj)
    rowsum = np.array(adj.sum(1))
    d_inv_sqrt = np.power(rowsum, -0.5).flatten()
    d_inv_sqrt[np.isinf(d_inv_sqrt)] = 0.
    d_mat_inv_sqrt = sp.diags(d_inv_sqrt)
    return adj.dot(d_mat_inv_sqrt).transpose().dot(d_mat_inv_sqrt).tocoo()

def create_and_normalize_adjacency(G):
    adj = nx.adjacency_matrix(G) # Obtains the adjacency matrix of the training graph
    #adj = normalize_adjacency(adj)
    adj = normalize_adj(adj)
    print('Created a normalized adjancency matrix of shape', adj.shape)
    indices = np.array(adj.nonzero()) # Gets the positions of non zeros of adj into indices
    print('Created indices', indices.shape, 'with the positions of non zeros in adj matrix')
    return adj, indices

def sparse_mx_to_torch_sparse_tensor(sparse_mx):
    sparse_mx = sparse_mx.tocoo().astype(np.float32)
    indices = torch.from_numpy(np.vstack((sparse_mx.row, sparse_mx.col)).astype(np.int64))
    values = torch.from_numpy(sparse_mx.data)
    shape = torch.Size(sparse_mx.shape)
    return torch.sparse.FloatTensor(indices, values, shape)


def text_to_list(text):
    return unidecode(text).split(',')

def intersection(lst1, lst2): # a function that returns the number of common items of two lists and 1 or 0 if there are common. This function will be used in add_authors_to_pairs to add this features to the pairs.
    lst3 = [value for value in lst1 if value in lst2]
    is_common = 1 if len(lst3)>0 else 0
    return len(lst3)+1, is_common+1


def add_authors_to_pairs (pairs, authors):
    authors = pd.DataFrame(authors)
    try: 
        pairs = pairs.detach().cpu().numpy()
    except:
        pass
        

    pairs_df = pd.DataFrame(np.transpose(pairs)).rename(columns={0: "paper_1", 1: "paper_2"})
    pairs_df = pairs_df.merge(authors, left_on='paper_1', right_on='paper_permut', how='left').rename(columns={'authors': "authors_1"})
    pairs_df = pairs_df.merge(authors, left_on='paper_2', right_on='paper_permut', how='left').rename(columns={'authors': "authors_2"})
    pairs_df.drop(['paper_id_x', 'paper_id_y'], axis=1, inplace=True)

    pairs_df['nb_common_author'] = pairs_df.apply(lambda row: intersection(row['authors_1'], row['authors_2'])[0], axis=1)
    pairs_df['is_common_author'] = pairs_df.apply(lambda row: intersection(row['authors_1'], row['authors_2'])[1], axis=1)

    pairs_tensor = torch.LongTensor(np.transpose(pairs_df[["paper_1", "paper_2", 'is_common_author', 'nb_common_author']].values.tolist())).to(device)
    
    return pairs_tensor


In [23]:
def read_and_clean_abstracts (nodes, sample_length=-1, abstracts_path = 'https://www.lix.polytechnique.fr/~nikolentzos/files/aai/challenge/abstracts.txt'):
    t = time()
    abstracts = dict()
    abstracts_list = list()
    f = urlopen(abstracts_path)
    
    for i, line in tqdm(enumerate(f)):
        if i == sample_length:
            break
        if i in nodes:
            node, abstract = str(line).lower().split('|--|')
            abstract = remove_stopwords(abstract)
            #abstract = re.sub(r"[,.;@#?!&$()-]", " ", abstract)
            abstract = re.sub(r"[^a-zA-Z0-9\s]", "", abstract)
            #abstract = re.sub(r"\\", " ", abstract)
            abstract = remove_stopwords(abstract)

            for word in abstract.split()[:-1]:
                #abstract = abstract.replace(word, stemmer.stem(word))
                abstract = abstract.replace(word, lemmatizer.lemmatize(lemmatizer.lemmatize(lemmatizer.lemmatize(lemmatizer.lemmatize(lemmatizer.lemmatize(lemmatizer.lemmatize(word), pos='s'), pos='a'), pos='n'), pos='v'), pos='r'))
            
            node = re.sub("[^0-9]", "", node)
            if i != int(node):
                print('i and node not the same', i, node)
            abstracts[int(node)] = abstract
            abstracts_list.append(abstract)
        
    print('Text loaded and cleaned in {:.0f} min'.format((time()-t)/60))
    return abstracts

def doc_counter (documents, word): #a function that return the number of documents containing a word
    counter = 0
    for i in documents:
        if word in documents[i]:
            counter += 1
    return counter



In [24]:
class Vocabulary:

    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {}
        self.word_occurrence = {}
        self.word2node = {}
        self.words_list = []
        self.sentences_list = []
        self.sentences_list_words = []
        self.num_words = 0
        self.num_sentences = 0
        self.longest_sentence = 0

    def add_word(self, word, node):
        if word not in self.word2index:
            # First entry of word into vocabulary
            self.words_list.append(word)
            self.word2index[word] = self.num_words
            self.word2count[word] = 1
            self.index2word[self.num_words] = word
            self.num_words += 1
            self.word_occurrence[word] = 1
            self.word2node[word] = [node]
        else:
            # Word exists; increase word count
            self.word2count[word] += 1
            self.word_occurrence[word] += 1
            if node not in self.word2node[word]:
                self.word2node[word].append(node)
            # self.num_words += 1
            
    def add_sentence(self, sentence, node):
        sentence_len = 0
        for word in sentence.split()[:-1]:
            sentence_len += 1
            self.add_word(word, node)
        if sentence_len > self.longest_sentence:
            # This is the longest sentence
            self.longest_sentence = sentence_len
        # Count the number of sentences
        self.num_sentences += 1
        self.sentences_list.append(sentence)
        self.sentences_list_words.append(sentence.split()[:-1])

    def to_word(self, index):
        return self.index2word[index]

    def to_index(self, word):
        return self.word2index[word]

    def words(self):
        return self.words_list



In [17]:
def prepare_data_to_train (features, authors, adj, indices, val_indices, y_val):
    
    print('Preparing the data for training...')
    
    t = time()
    
    y_val = torch.LongTensor(y_val).to(device)

    # Create class labels
    y = np.zeros(2*indices.shape[1])
    y[:indices.shape[1]] = 1 # Concatenated ones for edges indices and later in the model we add zeros for random indices.
    y = torch.LongTensor(y).to(device)
    
    features = torch.FloatTensor(features).to(device)
    
    indices = torch.LongTensor(indices).to(device)
    adj = sparse_mx_to_torch_sparse_tensor(adj).to(device)
    #tfidf_matrix = sparse_mx_to_torch_sparse_tensor(tfidf_matrix).to(device)
    
    # the function add_authors_to_pairs converts into torch tensors and sends to Device    
    val_indices = add_authors_to_pairs(val_indices, authors) #we add the authors to val_pairs
    indices = add_authors_to_pairs(indices, authors) #we add the authors to indices    
    rand_indices = np.random.randint(0, features.shape[0], (indices.shape[0],indices.shape[1]))# We take random indices each time we run an epoch
    rand_indices = add_authors_to_pairs(rand_indices, authors)

    pairs = torch.cat((indices, rand_indices), dim=1) # Concatenate the edges indices and random indices. 
    del(authors, indices, rand_indices)
    
    print('Data converted into torch tensors and authors added to indices in {:.0f} min'.format((time()-t)/60))

    return features, adj, pairs, y, val_indices, y_val 

In [383]:
def map_features_with_permutation(features, permutation):
    new_features = np.zeros((len(features), len(features[0])))
    for i in range(len(features)):
        new_features[i] = features[permutation[i]]
    return new_features

# Load graph and authors data from sources

In [369]:
G, G_train, train_edges, val_edges, val_indices, y_val, nodes, node_to_idx, mapping_permutation = read_train_val_graph(val_ratio=0.1)


Number of nodes: 138499 number of edges: 1091955 in the Complete set
Number of nodes: 138499 number of edges: 982730 in the Training set
len(nodes) 138499
Creating random val_edges...
Returned G_train, train_edges, val_edges, y_val, nodes and node_to_idx objects
Loaded from edgelist.txt and with a training validation split ratio = 0.1


In [372]:
adj, indices = create_and_normalize_adjacency(G_train)


/var/folders/pd/glcnl2497w5b6xn3p94tnwlr0000gn/T/ipykernel_91389/2709409960.py:154: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  adj = nx.adjacency_matrix(G) # Obtains the adjacency matrix of the training graph


Created a normalized adjancency matrix of shape (138499, 138499)
Created indices (2, 1965460) with the positions of non zeros in adj matrix


/var/folders/pd/glcnl2497w5b6xn3p94tnwlr0000gn/T/ipykernel_91389/2709409960.py:148: RuntimeWarning: divide by zero encountered in power
  d_inv_sqrt = np.power(rowsum, -0.5).flatten()


In [ ]:
mapping_permutation

In [385]:
authors = pd.read_csv(urlopen('https://www.lix.polytechnique.fr/~nikolentzos/files/aai/challenge/authors.txt'), sep = '|', header=None)
authors = authors.rename(columns={0: "paper_id", 2: "authors"})
authors['authors'] = authors['authors'].apply(text_to_list)
authors = authors[["paper_id", "authors"]]
authors = authors[authors['paper_id'] <= max(G.nodes())]
authors['paper_permut'] = permutation[authors['paper_id']]
authors.head()

,paper_id,authors,paper_permut
0,0,"[James H. Niblock, Jian-Xun Peng, Karen R. McM...",12798
1,1,"[Jian-Xun Peng, Kang Li, De-Shuang Huang]",11956
2,2,[J. Heikkila],110297
3,3,"[L. Teslic, B. Hartmann, O. Nelles, I. Skrjanc]",8849
4,4,"[Long Zhang, Kang Li, Er-Wei Bai, George W. Ir...",47674


# Read processed data

In [19]:
t = time()
from io import BytesIO

walks_url = 'https://storage.googleapis.com/link_prediction_processed_data/walks_wv.npy'
with urlopen(walks_url) as url:
    data = url.read()

# Create a seekable file-like object from the data
fileobj = BytesIO(data)

# Load the data from the file object
walks_wv = np.load(fileobj)
print('wv walks loaded from GCP in {:.0f} sec'.format(time()-t))
walks_wv.shape

wv walks loaded from GCP in 73 sec


(138499, 64)

In [ ]:
# Load the BART embedding torch tensor

# url = "https://storage.googleapis.com/link_prediction_processed_data/bart_embeddings.pt"
# response = requests.get(url)

# with open("bart_embeddings.pt", "wb") as f:
#     f.write(response.content)

# abstracts_bart_embeddings = torch.load('bart_embeddings.pt')


In [ ]:
# # Load the wv300 mean embedding
# url = 'https://storage.googleapis.com/link_prediction_processed_data/embedded_abstracts_local_wv300.npy'

# response = requests.get(url)

# with open('embedded_abstracts_local_wv300.npy', 'wb') as f:
#     f.write(response.content)

# # Load the numpy array from the saved file
# local_wv300_abstracts = np.load('embedded_abstracts_local_wv300.npy')

# local_wv300_abstracts.shape


In [ ]:
# # Load the goog300 mean embedding
# url = 'https://storage.googleapis.com/link_prediction_processed_data/embedded_abstracts_goog300.npy'

# response = requests.get(url)

# with open('embedded_abstracts_goog300.npy', 'wb') as f:
#     f.write(response.content)

# # Load the numpy array from the saved file
# goog300_abstracts = np.load('embedded_abstracts_goog300.npy')

# goog300_abstracts.shape


In [ ]:
# # Load TF-IDF matrix

# url = "https://storage.googleapis.com/link_prediction_processed_data/tfidf_matrix.npz"

# response = requests.get(url)

# with open("tfidf_matrix.npz", "wb") as f:
#     f.write(response.content)

# tfidf_matrix = sparse.load_npz("tfidf_matrix.npz")

# tfidf_matrix.shape

In [1]:
# Load words 192 embedding
import gzip
import pickle

with gzip.open('embedded_abstracts_dict_192array.pkl.gz', 'rb') as f:
    words_embedding_192 = pickle.load(f)

len(words_embedding_192)

138499

In [202]:
len(words_embedding_192)

138499

In [4]:
from tqdm.notebook import tqdm
import numpy as np

words_embedding_192_trunc128 = dict ()
for i in tqdm(range(len(words_embedding_192))):
    if len(words_embedding_192[i])>0:
        arr = np.zeros((128, 192))
        vec = words_embedding_192[i][:128, :]
        arr[:vec.shape[0],:] = vec
        words_embedding_192_trunc128[i] = torch.tensor(arr).to(device)
    else:
        words_embedding_192_trunc128[i] = []


  0%|          | 0/138499 [00:00<?, ?it/s]

In [8]:
tensor_list = []
i = 0
for i in tqdm(range(len(words_embedding_192_trunc128))):
    if len(words_embedding_192_trunc128[i])>0:
        tensor_list.append(words_embedding_192_trunc128[i])
    else:
        tensor_list.append(torch.zeros((128, 192)))


  0%|          | 0/138499 [00:00<?, ?it/s]

In [12]:
del(words_embedding_192_trunc128)

In [5]:
import gzip

filename = 'words_embedding_192_trunc128.pkl.gz'

# open the file in binary mode and write the dictionary to it, compressing the data with gzip
with gzip.open(filename, 'wb') as f:
    pickle.dump(words_embedding_192_trunc128, f)

In [41]:
tensor_list_float = []
for i, tensor in tqdm(enumerate(tensor_list)):
    tensor_list_float.append(tensor_list[i].float())
del(tensor_list)

0it [00:00, ?it/s]

NameError: name 'words_embedding_192_trunc128' is not defined

# Model

In [14]:

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=128, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv1d(in_channels=64, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool1d(kernel_size=2, stride=2)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
        self.fc1 = nn.Linear(1536, 128)
        self.fc2 = nn.Linear(128*2, 64)
        self.fc5 = nn.Linear(64, 2)
        self.dropout = nn.Dropout(0.2)
    
    def forward(self, x1, x2):
        x1 = self.conv1(x1)
        x2 = self.conv1(x2)

        x1 = self.relu(x1)
        x2 = self.relu(x2)

        x1 = self.pool(x1)
        x2 = self.pool(x2)

        x1 = self.conv2(x1)
        x2 = self.conv2(x2)

        x1 = self.relu(x1)
        x2 = self.relu(x2)

        x1 = self.pool(x1)
        x2 = self.pool(x2)

        x1 = x1.view(-1, 1536)
        x2 = x2.view(-1, 1536)

        x1 = self.fc1(x1)
        x2 = self.fc1(x2)

        x = torch.cat((x1, x2), dim=1)
        del(x2)

        x = self.fc2(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc5(x)

        return F.log_softmax(x, dim=1), x1



In [185]:
def train_CNN(model, learning_rate, x, pairs, y, val_indices, y_val, epochs, batch_size, run_number, window = 10):
    # Train model
    start_time = time()
    
    print('Initializing the optimizer with learning rate:', learning_rate)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate) #optimizer with halving learning rate in training

    today = datetime.today().strftime('%Y-%m-%d-%H:%M')
    list_loss_val = []
    list_loss_train = []
    
    
    halving_lr = 0 # counter of the number of halving lr
    patience = 16
    early_stopping = EarlyStopping(model, patience=patience, delta=0.1, path='checkpoint.pt')
    print('Start training...')

    num_batches = (len(pairs) + batch_size - 1) // batch_size
    total_loss = 0
    total_acc = 0
    num_pairs = pairs.shape[1]

    for epoch in range(epochs):
        t = time()
        optimizer.zero_grad()
        
        model_CNN.train()

        perm = np.random.permutation(num_pairs)

        for i in range(0, len(pairs), batch_size):
            pairs1 = pairs[0][perm[i: i+batch_size]].numpy()
            pairs2 = pairs[1][perm[i: i+batch_size]].numpy()
            print(type(pairs1), pairs1)
            y = y[perm[i: i+batch_size]]
            x1_batch = (x[pairs1])
            print(x1_batch)
            x2_batch = (x[pairs2])
            output, _ = model_CNN(x1_batch, x2_batch) # we run the model that gives the output.
            output.to(device)
            loss_train = F.nll_loss(output, y) # we are using nll_loss as loss to optimize, we store it in loss_train. We compare to y which is stable and contains the tag ones and zeros.
            acc_train = accuracy_score(torch.argmax(output, dim=1).detach().cpu().numpy(), y.cpu().numpy())# just to show it in the out put message of the training
            loss_train.backward() # The back propagation ? --> Computes the gradient of current tensor w.r.t. graph leaves
            optimizer.step() # Performs a single optimization step (parameter update).
            total_loss += loss_train.item()
            total_acc += acc_train
        loss_train = total_loss / num_batches
        acc_train = total_acc / num_batches
        
        model.eval()
        val_indices_torch = torch.Tensor(val_indices).long()
        
        x1 = x[val_indices[0].tolist()]
        x2 = x[val_indices[1].tolist()]
        output, _ = model_CNN(x1, x2)
        output.to(device)
        #y_val = torch.LongTensor(y_val).to(device)
        loss_val = F.nll_loss(output, y_val)
        list_loss_val.append(loss_val.item())
        list_loss_train.append(loss_train.item())
        acc_val = accuracy_score(torch.argmax(output, dim=1).detach().cpu().numpy(), y_val.cpu().numpy())
        


        if epoch % 5 == 0:
            print('Epoch: {:03d}'.format(epoch+1),
                  'loss_train: {:.4f}'.format(loss_train.item()),
                  'loss_val: {:.4f}'.format(loss_val.item()),
                  'acc_train: {:.4f}'.format(acc_train.item()),
                  'acc_val: {:.4f}'.format(acc_val.item()),
                  'time: {} s'.format(int(round(time()) - round(t))),
                 'total_time: {} min'.format(round((time() - start_time)/60)))
            break # <----- break to remove
            
            if epoch % 50 == 0:
                model_path = "outputs/{}-model-{}epochs-{}.pt".format(today, epoch, run_number)
                torch.save(model.state_dict(), model_path)
            

        
        early_stopping(loss_val)

        if early_stopping.early_stop:
            halving_lr += 1
            if halving_lr > 4:
                break
            learning_rate = learning_rate/10
            optimizer = optim.Adam(model.parameters(), lr=learning_rate)
            early_stopping = EarlyStopping(model, patience=patience, delta=0.01, path='checkpoint.pt')
            print('Deviding the learning rate by 10. New learning rate: {}'.format(learning_rate))



    print("Optimization Finished in {} min!".format(round((time() - start_time)/60)))
    return model







# for key, tensor in words_embedding_dict.items():
#     output_dict[key] = model(tensor.unsqueeze(0), pairs).squeeze()

# # Print the output shape of each tensor
# for key, tensor in output_dict.items():
#     print(key, tensor.shape)

In [265]:
class GNN(nn.Module):
    def __init__(self, n_text, n_feat, n_hidden, n_class, sub_class, dropout):
        super(GNN, self).__init__()
        #self.input_txt = nn.Linear(n_text, int(n_hidden))
        self.fc1 = nn.Linear(n_feat, n_hidden)        
        self.fc2 = nn.Linear(n_hidden, n_hidden)
        self.fc3 = nn.Linear(2*n_hidden, n_hidden)
        self.fc4 = nn.Linear(n_hidden, sub_class)
        self.fc5 = nn.Linear(sub_class, n_class)        
        self.dropout = nn.Dropout(dropout)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
        
        

    def forward(self, x_in, abstract, adj, pairs):

        # text shape: [batch_size, max_sent_len]

        # Embedding layer
        embedded = self.embedding(abstract) # shape: [batch_size, max_sent_len, embedding_dim]

        # Add a channel dimension for convolutional layers
        embedded = embedded.unsqueeze(1) # shape: [batch_size, 1, max_sent_len, embedding_dim]

        # Convolutional layers
        conved = [nn.functional.relu(conv(embedded)).squeeze(3) for conv in self.convs] # shape: [batch_size, num_filters, max_sent_len - filter_size + 1]

        # Max pooling
        pooled = [nn.functional.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved] # shape: [batch_size, num_filters]

        # Concatenate pooled outputs from different filter sizes
        cat = torch.cat(pooled, dim=1) # shape: [batch_size, num_filters * len(filter_sizes)]

        # Fully connected layer
        output = self.fc(cat) # shape: [batch_size, output_dim]

        # Sigmoid activation
        output = self.sigmoid(output) # shape: [batch_size, output_dim]
        
        
        
        h1 = self.fc1(x_in)
        z1 = self.relu(torch.spmm(adj, h1))
        z1 = self.dropout(z1)
        del(x_in, h1)

        h2 = self.fc2(z1)
        z2 = self.relu(torch.spmm(adj, h2))
        z2 = self.dropout(z2)
        del(h2, z1, adj)
        
        x = torch.cat((z2[pairs[0]] , z2[pairs[1]]), dim=1)
        del(z2)
        x = (pairs[2][:, None])*x       
        
        #x = torch.cat((x, y), dim=1)        
        del(pairs)

        x = self.relu(self.fc3(x))
        x = self.dropout(x)
        
        x = self.relu(self.fc4(x))
        x = self.dropout(x)
        
        x = self.fc5(x)
        return F.log_softmax(x, dim=1)




In [ ]:
import torch.nn.functional as F
from scipy.sparse import csr_matrix

class GNN(nn.Module):
    def __init__(self, n_text, n_feat, n_hidden, n_class, sub_class, dropout):
        super(GNN, self).__init__()
        self.input_txt = nn.Linear(n_text, int(n_hidden))
        self.conv1 = nn.Conv1d(in_channels=n_hidden, out_channels=n_hidden, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(n_feat, n_hidden)
        self.fc2 = nn.Linear(n_hidden, n_hidden)
        self.fc3 = nn.Linear(4*n_hidden, n_hidden)
        self.fc4 = nn.Linear(n_hidden, sub_class)
        self.fc5 = nn.Linear(sub_class, n_class)
        self.dropout = nn.Dropout(dropout)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x_in, tf_idf, adj, pairs):
        tf_idf_sparse = torch.sparse_coo_tensor(tf_idf._indices(), tf_idf._values(), tf_idf.size())
        y = nn.functional.relu(self.input_txt(tf_idf_sparse.to(device)))
        y = torch.transpose(y, 0, 1)
        y = nn.functional.relu(self.conv1(y))
        y = y.unsqueeze(dim=0)  # add a new dimension with size 1 instead of batch size
        y = torch.max_pool1d(y, kernel_size=y.shape[2])
        y = y.squeeze(dim=2)

        y = self.dropout(y)
        del(tf_idf)

        y = torch.cat((y[pairs[0]], y[pairs[1]]), dim=1)

        y = torch.cat((y[pairs[0]], y[pairs[1]]), dim=1)

        h1 = self.fc1(x_in)
        z1 = self.relu(torch.spmm(adj, h1)) # sparce matrix multiplication
        z1 = self.dropout(z1)
        del(x_in, h1)

        h2 = self.fc2(z1)
        z2 = self.relu(torch.spmm(adj, h2))
        z2 = self.dropout(z2)
        del(h2, z1, adj)

        x = torch.cat((z2[pairs[0]], z2[pairs[1]]), dim=1)
        del(z2)
        x = (pairs[2][:, None])*x

        x = torch.cat((x, y), dim=1)

        del(pairs)

        x = self.relu(self.fc3(x))
        x = self.dropout(x)

        x = self.relu(self.fc4(x))
        x = self.dropout(x)

        x = self.fc5(x)

        return F.log_softmax(x, dim=1)




In [391]:


class EarlyStopping:
    def __init__(self, model, patience, delta, path='checkpoint.pt'):
        self.patience = patience
        self.delta = delta
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.model = model
        self.val_loss_min = np.Inf
        
    def __call__(self, val_loss, path='checkpoint.pt'):
        score = val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(path)
        elif score > self.best_score + 0:
            self.counter += 1
            #print(self.counter)
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(path)
            self.counter = 0

    def save_checkpoint(self, path):
        torch.save(self.model.state_dict(), path)

    def load_checkpoint(self, path):
        self.model.load_state_dict(torch.load(path))



def train_model(model, learning_rate, bart_abstract, features, adj, pairs, y, val_indices, y_val, epochs, run_number, window = 10):
    # Train model
    start_time = time()
    
    print('Initializing the optimizer with learning rate:', learning_rate)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate) #optimizer with halving learning rate in training
    try: os.mkdir('./outputs')
    except: pass

    today = datetime.today().strftime('%Y-%m-%d-%H:%M')
    list_loss_val = []
    list_loss_train = []
    list_epochs = []
    
    halving_lr = 0 # counter of the number of halving lr
    patience = 16
    early_stopping = EarlyStopping(model, patience=patience, delta=0.1, path='checkpoint.pt')
    print('Start training...')
    for epoch in range(epochs):
        t = time()
        optimizer.zero_grad()
        
        model.train()

        output = model(features, bart_abstract, adj, pairs).to(device) # we run the model that gives the output.
        loss_train = F.nll_loss(output, y) # we are using nll_loss as loss to optimize, we store it in loss_train. We compare to y which is stable and contains the tag ones and zeros.
        acc_train = accuracy_score(torch.argmax(output, dim=1).detach().cpu().numpy(), y.cpu().numpy())# just to show it in the out put message of the training
        loss_train.backward() # The back propagation ? --> Computes the gradient of current tensor w.r.t. graph leaves
        optimizer.step() # Performs a single optimization step (parameter update).
        
        model.eval()
        output = model(features, bart_abstract, adj, val_indices).to(device)
        #y_val = torch.LongTensor(y_val).to(device)
        loss_val = F.nll_loss(output, y_val)
        list_loss_val.append(loss_val.item())
        list_loss_train.append(loss_train.item())
        list_epochs.append(epoch)
        acc_val = accuracy_score(torch.argmax(output, dim=1).detach().cpu().numpy(), y_val.cpu().numpy())
        


        if epoch % 5 == 0:
            print('Epoch: {:03d}'.format(epoch+1),
                  'loss_train: {:.4f}'.format(loss_train.item()),
                  'loss_val: {:.4f}'.format(loss_val.item()),
                  'acc_train: {:.4f}'.format(acc_train.item()),
                  'acc_val: {:.4f}'.format(acc_val.item()),
                  'time: {} s'.format(int(round(time()) - round(t))),
                 'total_time: {} min'.format(round((time() - start_time)/60)))
            
            if epoch % 50 == 0:
                model_path = "outputs/{}-model-{}epochs-{}.pt".format(today, epoch, run_number)
                torch.save(model.state_dict(), model_path)
            

        
        early_stopping(loss_val)

        if early_stopping.early_stop:
            halving_lr += 1
            if halving_lr > 4:
                break
            learning_rate = learning_rate/10
            optimizer = optim.Adam(model.parameters(), lr=learning_rate)
            early_stopping = EarlyStopping(model, patience=patience, delta=0.01, path='checkpoint.pt')
            print('Deviding the learning rate by 10. New learning rate: {}'.format(learning_rate))



    print("Optimization Finished in {} min!".format(round((time() - start_time)/60)))
    return model, list_loss_val, list_loss_train, list_epochs



# Train the model

In [384]:
# shuffle features the same way as the Graph
walks_wv = map_features_with_permutation(walks_wv, permutation)

In [388]:
features_torch, adj_torch, pairs_torch, y_torch, val_indices_torch, y_val_torch = prepare_data_to_train(walks_wv, authors, adj, indices, val_indices, y_val)

#abstracts_bart_embeddings = abstracts_bart_embeddings.to(device)
#goog300_abstracts_torch = torch.FloatTensor(goog300_abstracts).to(device)
#local_wv300_abstracts_torch = torch.FloatTensor(local_wv300_abstracts).to(device)
#tfidf_matrix_torch = sparse_mx_to_torch_sparse_tensor(tfidf_matrix).to(device)

Preparing the data for training...
Data converted into torch tensors and authors added to indices in 2 min


In [389]:
n_hidden = 64
dropout_rate = 0.2
sub_class = 8
n_class = 2
text_embedding = 0 #tfidf_matrix_torch
n_text = 1 #text_embedding.shape[1]
n_features = features_torch.shape[1]


# Creates the model
model = GNN(n_text, n_features, n_hidden, n_class, sub_class, dropout_rate).to(device)


In [390]:
torch.cuda.empty_cache()


epochs = 602
run_number = randint(0, 1000)
learning_rate = 0.01


trained_model, model, list_loss_val, list_loss_train, list_epochs = train_model(model, learning_rate, text_embedding, 
                            features_torch, adj_torch, pairs_torch, 
                            y_torch, val_indices_torch, y_val_torch, epochs, run_number)


Initializing the optimizer with learning rate: 0.01
Start training...
Epoch: 001 loss_train: 0.7009 loss_val: 0.6841 acc_train: 0.5000 acc_val: 0.5293 time: 11 s total_time: 0 min
Epoch: 006 loss_train: 0.6495 loss_val: 0.6490 acc_train: 0.6827 acc_val: 0.7336 time: 8 s total_time: 1 min
Epoch: 011 loss_train: 0.6215 loss_val: 0.6100 acc_train: 0.6269 acc_val: 0.7275 time: 6 s total_time: 1 min
Epoch: 016 loss_train: 0.6153 loss_val: 0.5919 acc_train: 0.6681 acc_val: 0.6335 time: 7 s total_time: 2 min
Epoch: 021 loss_train: 0.5680 loss_val: 0.5529 acc_train: 0.7346 acc_val: 0.7575 time: 9 s total_time: 3 min
Epoch: 026 loss_train: 0.5314 loss_val: 0.5209 acc_train: 0.7645 acc_val: 0.7716 time: 7 s total_time: 3 min
Epoch: 031 loss_train: 0.5204 loss_val: 0.5235 acc_train: 0.7538 acc_val: 0.7536 time: 7 s total_time: 4 min
Epoch: 036 loss_train: 0.5043 loss_val: 0.4955 acc_train: 0.7643 acc_val: 0.7762 time: 7 s total_time: 4 min
Epoch: 041 loss_train: 0.5014 loss_val: 0.4945 acc_train:

KeyboardInterrupt: 

In [348]:
torch.cuda.empty_cache()


epochs = 602
run_number = randint(0, 1000)
learning_rate = 0.01


trained_model = train_model(model, learning_rate, text_embedding, 
                            features_torch, adj_torch, pairs_torch, 
                            y_torch, val_indices_torch, y_val_torch, epochs, run_number)


Initializing the optimizer with learning rate: 0.01
Start training...
Epoch: 001 loss_train: 0.7602 loss_val: 0.7479 acc_train: 0.5000 acc_val: 0.5000 time: 12 s total_time: 0 min
Epoch: 006 loss_train: 0.7065 loss_val: 0.6908 acc_train: 0.5000 acc_val: 0.5000 time: 7 s total_time: 1 min
Epoch: 011 loss_train: 0.6521 loss_val: 0.6157 acc_train: 0.5457 acc_val: 0.5889 time: 7 s total_time: 1 min
Epoch: 016 loss_train: 0.5959 loss_val: 0.5776 acc_train: 0.6227 acc_val: 0.7131 time: 6 s total_time: 2 min
Epoch: 021 loss_train: 0.5506 loss_val: 0.5312 acc_train: 0.7415 acc_val: 0.7647 time: 7 s total_time: 2 min
Epoch: 026 loss_train: 0.5185 loss_val: 0.4993 acc_train: 0.7520 acc_val: 0.7719 time: 6 s total_time: 3 min
Epoch: 031 loss_train: 0.4862 loss_val: 0.4654 acc_train: 0.7779 acc_val: 0.7934 time: 6 s total_time: 4 min
Epoch: 036 loss_train: 0.4516 loss_val: 0.4313 acc_train: 0.7880 acc_val: 0.8063 time: 7 s total_time: 4 min
Epoch: 041 loss_train: 0.4239 loss_val: 0.4104 acc_train:

KeyboardInterrupt: 

In [337]:
# before shuffling
torch.cuda.empty_cache()


epochs = 602
run_number = randint(0, 1000)
learning_rate = 0.01


trained_model = train_model(model, learning_rate, text_embedding, 
                            features_torch, adj_torch, pairs_torch, 
                            y_torch, val_indices_torch, y_val_torch, epochs, run_number)


Initializing the optimizer with learning rate: 0.01
Start training...
Epoch: 001 loss_train: 0.6945 loss_val: 0.6931 acc_train: 0.4038 acc_val: 0.5021 time: 14 s total_time: 0 min
Epoch: 006 loss_train: 0.6557 loss_val: 0.6425 acc_train: 0.5295 acc_val: 0.7204 time: 7 s total_time: 1 min
Epoch: 011 loss_train: 0.6177 loss_val: 0.5952 acc_train: 0.6884 acc_val: 0.6996 time: 8 s total_time: 1 min
Epoch: 016 loss_train: 0.5748 loss_val: 0.5468 acc_train: 0.7156 acc_val: 0.7352 time: 7 s total_time: 2 min
Epoch: 021 loss_train: 0.5282 loss_val: 0.4967 acc_train: 0.7502 acc_val: 0.7696 time: 7 s total_time: 3 min
Epoch: 026 loss_train: 0.4833 loss_val: 0.4552 acc_train: 0.7722 acc_val: 0.7913 time: 7 s total_time: 3 min
Epoch: 031 loss_train: 0.4514 loss_val: 0.4271 acc_train: 0.7864 acc_val: 0.8060 time: 8 s total_time: 4 min
Epoch: 036 loss_train: 0.4298 loss_val: 0.4141 acc_train: 0.7952 acc_val: 0.8051 time: 6 s total_time: 5 min
Epoch: 041 loss_train: 0.4111 loss_val: 0.4040 acc_train:

KeyboardInterrupt: 

In [271]:
np.shape(pairs_torch), y_torch.shape, val_indices_torch.shape, y_val_torch.shape

(torch.Size([4, 3931040]),
 torch.Size([3931040]),
 torch.Size([4, 109195]),
 torch.Size([982760]))

In [ ]:
torch.cuda.empty_cache()


epochs = 602
run_number = randint(0, 1000)
learning_rate = 0.01


trained_model = train_model(model, learning_rate, text_embedding, 
                            features_torch, adj_torch, pairs_torch, 
                            y_torch, val_indices_torch, y_val_torch, epochs, run_number)


In [ ]:
break

In [ ]:
# with dense2 1024 of BART
torch.cuda.empty_cache()


epochs = 200
run_number = randint(0, 1000)
learning_rate = 0.005


trained_model = train_model(model, learning_rate, abstracts_bart_embeddings, features_torch, adj_torch, pairs_torch, y_torch, val_indices_torch, y_val_torch, epochs, run_number)


In [ ]:
break

In [ ]:
# Without dense 1000
torch.cuda.empty_cache()


epochs = 200
run_number = randint(0, 1000)


trained_model = train_model(model, 0.01, tfidf_matrix_torch, features_torch, adj_torch, pairs_torch, y_torch, val_indices_torch, y_val_torch, epochs, run_number)


# Generate test file

In [343]:
from datetime import datetime
import re

test_path = 'https://www.lix.polytechnique.fr/~nikolentzos/files/aai/challenge/test.txt'
node_pairs = list()
f = urlopen(test_path)

for line in f:
    t = str(line).split(',')
    t[0] = int(re.sub("[^0-9]", "", t[0]))
    t[1] = int(re.sub("[^0-9]", "", t[1]))
    node_pairs.append((node_to_idx[int(t[0])], node_to_idx[int(t[1])]))

node_pairs = np.transpose(node_pairs)
node_pairs = add_authors_to_pairs(node_pairs, authors)
#node_pairs = torch.LongTensor(node_pairs).to(device)

adj_torch = sparse_mx_to_torch_sparse_tensor(adj).to(device)
features_torch = torch.FloatTensor(walks_wv).to(device)

test_output = model(features_torch, text_embedding, adj_torch, node_pairs)
y_pred = torch.exp(test_output)
y_pred = y_pred.detach().cpu().numpy()

y_pred_true = list()
for element in y_pred:
    y_pred_true.append(element[1])

today = datetime.today().strftime('%Y-%m-%d')
random_nb = randint(0, 1000)
model_nb = 4

pd.DataFrame(y_pred_true, columns=['predicted']).to_csv(
"{}-submission-{}-{}.csv".format(today, model_nb, random_nb), header=True, index=True, index_label='id'
)

In [ ]:
features.shape

# BART

In [ ]:
import torch
from transformers import BartTokenizer, BartModel

# Load the BART model and tokenizer
model = BartModel.from_pretrained('facebook/bart-large')
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large')



In [ ]:
abstracts_bart_embeddings = []

# Define a function to generate embeddings for text
def get_bart_embeddings(text, model):
    # Tokenize the input text
    encoded_input = tokenizer(text, padding=True, truncation=True, return_tensors='pt')

    # Generate embeddings using the BART model
    with torch.no_grad():
        model_output = model(**encoded_input)
        embeddings = model_output.last_hidden_state.mean(dim=1).squeeze()

    return embeddings

In [ ]:
for i in tqdm(range(len(abstracts_bart_embeddings), len(voc.sentences_list))):
    abstract = voc.sentences_list[i]
    abstracts_bart_embeddings.append(get_bart_embeddings(abstract, model))


In [ ]:
saved_abstracts_bart_embeddings = abstracts_bart_embeddings

In [ ]:
len(saved_abstracts_bart_embeddings), len(abstracts_bart_embeddings), len(voc.sentences_list)

In [ ]:
abstracts_bart_embeddings = torch.stack(abstracts_bart_embeddings)
abstracts_bart_embeddings.shape

In [ ]:
abstracts_bart_embeddings.to(device)

In [ ]:
torch.save(abstracts_bart_embeddings, 'bart_embeddings.pt')


#Draft

In [ ]:
# Testing
model.eval()
node_pairs = np.array(np.transpose(node_pairs))
node_pairs = torch.LongTensor(node_pairs).to(device)

test_output = model(features, adj, node_pairs)
y_pred = torch.exp(test_output)
y_pred = y_pred.detach().cpu().numpy()

y_pred_true = list()
for element in y_pred:
    y_pred_true.append(element[1])
    

    
today = datetime.today().strftime('%Y-%m-%d')
random_nb = randint(0, 1000)

pd.DataFrame(y_pred_true, columns={'predicted'}).to_csv(
"{}-submission-{}-{}.csv".format(today, model_nb, random_nb), header=True, index=True, index_label='id'
)

In [ ]:
#### New script with batches

def early_stopping(loss_train, list_loss_train, loss_val, list_loss_val, 
                   tolerance=0.01, patience=15):
    list_loss_val = list(list_loss_val)[-patience:]
    list_loss_train = list(list_loss_train)[-patience:]
    if (len(list_loss_val) == patience and loss_val > (sum(list_loss_val)/len(list_loss_val)) and loss_train + tolerance < loss_val) or (len(list_loss_train) == patience and loss_train > (sum(list_loss_train)/len(list_loss_train))):
        #print('train: {:.5f} val: {:.5f} mean val: {:.5f}'.format(loss_train, loss_val, (sum(list_loss_val)/len(list_loss_val))))
        return True
    return False
    

    
def train_model(model, learning_rate, features, adj, indices_mc, y, val_indices, 
                y_val, epochs, batch_size, wv_walk_size, 
                tolerence = 0.01, patience = 15, run_number=randint(0, 1000)):
    # Train model
    start_time = time()
    print('Initializing the optimizer with learning rate:', learning_rate)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate) #optimizer with halving learning rate in training
    try: os.mkdir('./outputs')
    except: pass
    print('Preparing the data for training...')        

    today = datetime.today().strftime('%Y-%m-%d-%H:%M')
    list_loss_val = []
    list_loss_train = []

    
    halving_lr = 0 # counter of the number of halving lr
    print('Start training...')
    for epoch in range(epochs):
        t = time()

        # we create the rand indices corresponding to non edges (their y = 0)
        # we could apply a condition on epoch to run rand_indices (for speed purposes)
        rand_indices = np.random.randint(0, len(indices_mc), size=(indices_mc.shape[0], indices_mc.shape[1]))
        rand_indices = add_authors_to_pairs(rand_indices, authors)
        pairs = np.concatenate((indices_mc, rand_indices), axis=1)
        pairs = torch.LongTensor(pairs).to(device)

        permutation = torch.randperm(pairs.size()[1])
        
        # batches
        for i in range(0, pairs.size()[1], batch_size):
            optimizer.zero_grad()

            elts_indices = permutation[i:i+batch_size]
            batch_pairs = pairs[:, elts_indices]
            batch_y = y[elts_indices]

        
            model.train()

            output = model(features, adj, batch_pairs, wv_walk_size).to(device) # we run the model that gives the output.
            loss_train = F.nll_loss(output, batch_y) # we are using nll_loss as loss to optimize, we store it in loss_train. We compare to y which is stable and contains the tag ones and zeros.
            acc_train = accuracy_score(torch.argmax(output, dim=1).detach().cpu().numpy(), batch_y.cpu().numpy())# just to show it in the out put message of the training
            loss_train.backward() # The back propagation ? --> Computes the gradient of current tensor w.r.t. graph leaves
            optimizer.step() # Performs a single optimization step (parameter update).
        
        model.eval()
        output = model(features, adj, val_indices, wv_walk_size).to(device)
        #y_val = torch.LongTensor(y_val).to(device)
        loss_val = F.nll_loss(output, y_val)
        list_loss_val.append(loss_val.item())
        list_loss_train.append(loss_train.item())
        acc_val = accuracy_score(torch.argmax(output, dim=1).detach().cpu().numpy(), y_val.cpu().numpy())
        
        if epoch % 5 == 0:
            print('Epoch: {:03d}'.format(epoch+1),
                  'loss_train: {:.4f}'.format(loss_train.item()),
                  'loss_val: {:.4f}'.format(loss_val.item()),
                  'acc_train: {:.4f}'.format(acc_train.item()),
                  'acc_val: {:.4f}'.format(acc_val.item()),
                  'time: {} s'.format(int(round(time()) - round(t))),
                 'total_time: {} min'.format(round((time() - start_time)/60)))
        if epoch % 50 == 0:
            model_path = "outputs/{}-model-{}epochs-{}.pt".format(today, epoch, run_number)
            torch.save(model.state_dict(), model_path)
        
        if int(loss_val.item()) > 5:
            break
            
        early = early_stopping(loss_train.item(), list_loss_train, loss_val.item(), list_loss_val, patience=15)        
        if early:
            halving_lr += 1
            if halving_lr > 5:
                break
            list_loss_val=[]
            learning_rate = learning_rate/10
            optimizer = optim.Adam(model.parameters(), lr=learning_rate)
            print('Deviding the learning rate by 2. New learning rate: {:.6f}'.format(learning_rate))



    print("Optimization Finished in {} min!".format(round((time() - start_time)/60)))
    return model



In [ ]:
epochs = 1000

trained_model = train_model(model, 0.01, features, authors, adj, indices, y, torch.tensor(val_indices).to(device), torch.tensor(y_val).to(device), epochs)
